In [1]:
# 1) 环境与引擎
import os
print('QWEN_API_KEY set:', bool(os.getenv('QWEN_API_KEY')))
print('GOOGLE_API_KEY set:', bool(os.getenv('GOOGLE_API_KEY')))
# 默认本地模式，可通过 NB_ENGINE 覆盖为 qwen/gemini
if not os.getenv('NB_ENGINE'):
    os.environ['NB_ENGINE'] = 'local'

QWEN_API_KEY set: True
GOOGLE_API_KEY set: False


### 🔬 FlowDevExtractor 流动化学提取工具 - 调试版本


In [2]:
## 📦 导入必要的库和模块
import pandas as pd
import os
import sys
import shutil
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# 2) 导入
from core.text_utils import extract_text_from_pdf, write_text
from core.embedding import run_embedding_selection
from core.models.qwen_llm import QwenLLM
from core.models.gemini_llm import GeminiLLM
from core.processor import UnifiedTextProcessor

print("✅ 模块导入成功！")


/Users/zhaowenyuan/miniconda3/envs/ossextractor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 模块导入成功！


In [3]:
os.environ['FCPD_TOP_N'] = '15' # 候选段数 微调为20，未微调为15
os.environ['FCPD_EMB_KEEP_ORDER'] = '1' #确保“轻拼接后的写出顺序也可按原文顺序输出”
# 嵌入阶段对短段落做“轻拼接”，提高信息量。
os.environ['FCPD_EMB_EXPAND'] = '1'      # 1开启合并（默认关）
os.environ['FCPD_EMB_MIN_CHARS'] = '300' # 小于此长度会尝试合并
os.environ['FCPD_EMB_MAX_CHARS'] = '1000'# 合并后上限，避免超长
# 在段落抽取前“重拼接”相邻段，充分利用4096上下文。
# os.environ['FCPD_SUM_PACK'] = '0'  # 未微调模式
os.environ['FCPD_SUM_PACK'] = '1'  # 0是关闭合并 微调模式
os.environ['FCPD_SUM_PACK_MIN_CHARS'] = '500'
os.environ['FCPD_SUM_PACK_MAX_CHARS'] = '1500'    
# 增强标题检测窗口
os.environ['FCPD_FILTER_TITLE_WINDOW'] = '100'
# 关闭推理
os.environ['FCPD_RUN_IMPACT'] = '0' 
os.environ['FCPD_IMPACT_SOURCE'] = 'abstract' #从 Abstract 提取 Impact”

os.environ['FCPD_OVERALL_MAX_CAND'] = '20'
os.environ['FCPD_OVERALL_CHAR_BUDGET'] = '7000'
os.environ['FCPD_OVERALL_CHUNK_SIZE'] = '10'
# os.environ['FCPD_OVERALL_MAX_TOKENS'] = '1024' # Overall 阶段可以适当降低输出上限（因为未微调模型通常输出较短）
os.environ['FCPD_OVERALL_MAX_TOKENS'] = '4096' #微调模式

# 强制激活微调模式逻辑
os.environ['FCPD_FORCE_FINETUNED'] = '1'
# 开启调试日志以确认
os.environ['FCPD_DEBUG'] = '1'

### 简要流程
- 本地 LLM（默认）：五步法（PDF→TXT → Embedding → Filter → Abstract → Summarize/Overall/Impact）
- 在线 LLM（Qwen/Gemini）：直达模式（PDF→TXT → Fast Overall/Impact）


In [ ]:
### 🔧 配置参数
import os

# 配置要处理的PDF文件
pdf_files = [
    # 'evaluation/raw papers/0.pdf',
    'evaluation/raw papers/1.pdf',
    'evaluation/raw papers/2.pdf',
    'evaluation/raw papers/3.pdf',
    'evaluation/raw papers/4.pdf',
    'evaluation/raw papers/5.pdf',
    'evaluation/raw papers/6.pdf',
    'evaluation/raw papers/7.pdf',
    'evaluation/raw papers/8.pdf',
    'evaluation/raw papers/9.pdf',
    'evaluation/raw papers/10.pdf',
    # 如果有更多文件，可以加在这里
]

# 定义基础的数据目录
base_data_dir = 'evaluation/result'

# 1. 在Data目录下，定义一个名为 'output' 的主输出文件夹路径
main_output_dir = os.path.join(base_data_dir, 'local llm finetuned')
# main_output_dir = os.path.join(base_data_dir, 'local llm unfinetuned')

# 2. 创建 'output' 文件夹 (如果它不存在的话)
# exist_ok=True 表示如果文件夹已存在，则不会报错
os.makedirs(main_output_dir, exist_ok=True)

print(f"📄 将处理 {len(pdf_files)} 个PDF文件:")
print(f"📁 主输出目录已设置为: {main_output_dir}")
print("-" * 40) # 打印分割线

# 遍历每一个要处理的PDF文件
for i, pdf_path in enumerate(pdf_files, 1):
    
    # 3. 从完整路径中获取PDF的文件名 (例如: 'd2cp03073j.pdf')
    pdf_filename = os.path.basename(pdf_path)
    
    # 4. 去掉.pdf扩展名，创建文件夹名 (例如: 'd2cp03073j')
    folder_name = os.path.splitext(pdf_filename)[0]
    
    # 5. 拼接出这个PDF专属的输出文件夹的完整路径
    specific_output_dir = os.path.join(main_output_dir, folder_name)
    
    # 6. 创建这个专属的文件夹
    os.makedirs(specific_output_dir, exist_ok=True)
    
    print(f"  {i}. 正在处理: {pdf_filename}")
    print(f"     -> 将输出到: {specific_output_dir}")
    # --- 在这里接上你后续的处理逻辑 ---
    # 例如，你之后所有保存文件的操作，都应该使用 `specific_output_dir` 作为路径
    # processed_text_path = os.path.join(specific_output_dir, 'Processed_text.txt')
    # with open(processed_text_path, 'w') as f:
    #     f.write("这里是处理后的文本")


📄 将处理 10 个PDF文件:
📁 主输出目录已设置为: evaluation/result/local llm finetuned
----------------------------------------
  1. 正在处理: 0.pdf
     -> 将输出到: evaluation/result/local llm finetuned/0
  2. 正在处理: 1.pdf
     -> 将输出到: evaluation/result/local llm finetuned/1
  3. 正在处理: 2.pdf
     -> 将输出到: evaluation/result/local llm finetuned/2
  4. 正在处理: 3.pdf
     -> 将输出到: evaluation/result/local llm finetuned/3
  5. 正在处理: 5.pdf
     -> 将输出到: evaluation/result/local llm finetuned/5
  6. 正在处理: 6.pdf
     -> 将输出到: evaluation/result/local llm finetuned/6
  7. 正在处理: 7.pdf
     -> 将输出到: evaluation/result/local llm finetuned/7
  8. 正在处理: 8.pdf
     -> 将输出到: evaluation/result/local llm finetuned/8
  9. 正在处理: 9.pdf
     -> 将输出到: evaluation/result/local llm finetuned/9
  10. 正在处理: 10.pdf
     -> 将输出到: evaluation/result/local llm finetuned/10




你可以在这里为不同的处理阶段指定使用不同的LLM模型。


In [5]:
### 模型配置 - 分阶段自定义
"""
🎛️ 引擎初始化（单次）
支持引擎: local | qwen | gemini
优先读取 config.yaml；若 NB_ENGINE 设置且合法则覆盖。
"""

import yaml

# 读取配置
with open('config.yaml', 'r', encoding='utf-8') as f:
    _cfg = yaml.safe_load(f) or {}

_engine_cfg = (_cfg.get('engine', 'local') or 'local').strip().lower()
_engine_env = (os.getenv('NB_ENGINE') or '').strip().lower()
ENGINE = _engine_env if _engine_env in ['local','qwen','gemini'] else _engine_cfg

# 默认限制嵌入TOP_N，避免内存/显存压力
if not os.getenv('FCPD_TOP_N'):
    os.environ['FCPD_TOP_N'] = '10'

# 初始化处理器实例（单次）
if ENGINE == 'local':
    from core.local_pipeline import LocalPipeline
    lc = _cfg.get('local_model', {})
    engine = LocalPipeline(
        model_name=None,
        model_path=lc.get('path', 'models/'),
        filter_model=lc.get('filter'),
        abstract_model=lc.get('abstract'),
        summarize_model=lc.get('summarize'),
        overall_model=lc.get('overall'),
        impact_model=lc.get('impact'),
        finetuned_trigger_name=lc.get('finetuned_trigger_name', 'My_Finetuned_Model'),
    )
    ENGINE_NAME = 'local llm'
    # 判定本地子路径（未微调/微调）
    try:
        _sn = getattr(engine, 'model_summarize_name', None) or ''
        _on = getattr(engine, 'model_overall_name', None) or ''
        _is_ft = (_sn and engine._is_finetuned_model(_sn)) or (_on and engine._is_finetuned_model(_on))
        LOCAL_SUBPATH = 'local+finetune' if _is_ft else 'local (classic)'
    except Exception:
        LOCAL_SUBPATH = 'local (classic)'
elif ENGINE == 'qwen':
    q = _cfg.get('qwen_api', {})
    llm = QwenLLM(api_key_env_var=q.get('api_key_env_var', 'QWEN_API_KEY'), 
                  model_name=q.get('model_name', 'qwen-plus'))
    engine = UnifiedTextProcessor(llm)
    ENGINE_NAME = 'qwen'
    LOCAL_SUBPATH = None
else:
    # Gemini
    g = _cfg.get('gemini_api', {})
    llm = GeminiLLM(api_key_env_var=g.get('api_key_env_var', 'GOOGLE_API_KEY'), 
                    model_name=g.get('model_name', 'gemini-1.5-flash'))
    engine = UnifiedTextProcessor(llm)
    ENGINE_NAME = 'gemini'
    LOCAL_SUBPATH = None

print("=" * 60)
print("🎛️  引擎配置")
print("=" * 60)
print(f"📌 引擎:          {ENGINE}")
print(f"📌 归档目录名:     {ENGINE_NAME}")
if ENGINE == 'local':
    print(f"📌 本地子路径:     {LOCAL_SUBPATH}")
print("=" * 60)
print("✅ 处理器初始化完成（不会重复实例化）")


  [Debug] LocalPipeline Init: Summarize='My_Finetuned_Model_Q4_K_M_Mixed.gguf', Overall='My_Finetuned_Model_Q4_K_M_Mixed.gguf'
  [Debug] Finetuned Trigger: 'My_Finetuned_Model'
🎛️  引擎配置
📌 引擎:          local
📌 归档目录名:     local llm
📌 本地子路径:     local+finetune
✅ 处理器初始化完成（不会重复实例化）


## 📄 步骤 1: PDF转文本处理


In [6]:
print("🚀 步骤 1/5: PDF转文本处理...")
print("=" * 50)

# 直接使用新工具将PDF转为txt
output_files = []
for pdf in pdf_files:
    base = os.path.splitext(os.path.basename(pdf))[0]
    out_dir = os.path.join(main_output_dir, base)
    os.makedirs(out_dir, exist_ok=True)
    out_txt = os.path.join(out_dir, f"{base}.txt")
    text = extract_text_from_pdf(pdf)
    write_text(text, out_txt)
    output_files.append(out_txt)

# # 归档到引擎目录（与main一致）
# engine_name = ENGINE_NAME  # 由引擎初始化单元确定
# for txt in output_files:
#     base = os.path.splitext(os.path.basename(txt))[0]
#     dest_dir = os.path.join(main_output_dir, '..', engine_name, base)
#     os.makedirs(dest_dir, exist_ok=True)
#     try:
#         shutil.copy2(txt, os.path.join(dest_dir, os.path.basename(txt)))
#     except Exception:
#         pass

print(f"✅ PDF转文本完成并已归档！生成了 {len(output_files)} 个文本文件:")
for i, file in enumerate(output_files, 1):
    print(f"  {i}. {file}")
    if os.path.exists(file):
        with open(file, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
            print(f"     📊 行数: {len(lines)}")
            print(f"     📏 文件大小: {os.path.getsize(file)} bytes")


🚀 步骤 1/5: PDF转文本处理...
✅ PDF转文本完成并已归档！生成了 10 个文本文件:
  1. evaluation/result/local llm finetuned/0/0.txt
     📊 行数: 137
     📏 文件大小: 32888 bytes
  2. evaluation/result/local llm finetuned/1/1.txt
     📊 行数: 253
     📏 文件大小: 22013 bytes
  3. evaluation/result/local llm finetuned/2/2.txt
     📊 行数: 251
     📏 文件大小: 46012 bytes
  4. evaluation/result/local llm finetuned/3/3.txt
     📊 行数: 277
     📏 文件大小: 31911 bytes
  5. evaluation/result/local llm finetuned/5/5.txt
     📊 行数: 263
     📏 文件大小: 42726 bytes
  6. evaluation/result/local llm finetuned/6/6.txt
     📊 行数: 1169
     📏 文件大小: 37606 bytes
  7. evaluation/result/local llm finetuned/7/7.txt
     📊 行数: 159
     📏 文件大小: 18723 bytes
  8. evaluation/result/local llm finetuned/8/8.txt
     📊 行数: 83
     📏 文件大小: 21628 bytes
  9. evaluation/result/local llm finetuned/9/9.txt
     📊 行数: 1031
     📏 文件大小: 36427 bytes
  10. evaluation/result/local llm finetuned/10/10.txt
     📊 行数: 173
     📏 文件大小: 29261 bytes


### 🔍 查看PDF转文本结果


In [7]:
# # 选择第一个文件进行详细查看
# sample_file = output_files[0]
# print(f"📖 查看文件: {os.path.basename(sample_file)}")
# print("=" * 50)

# with open(sample_file, 'r', encoding='utf-8', errors='ignore') as f:
#     content = f.read()
    
# print(f"📊 总字符数: {len(content)}")
# print(f"📊 总行数: {len(content.splitlines())}")
# print("\n📄 前500个字符预览:")
# print("-" * 30)
# print(content[:500] + "..." if len(content) > 500 else content)


## 📝 步骤 2: 文本预处理


In [8]:
print("🚀 步骤 2/5: 文本预处理...")
print("=" * 50)

# 新架构：预处理整合到后续流程，直接使用原txt即可
processed_files = list(output_files)
print(f"✅ 预处理完成（直通）。文件数: {len(processed_files)}")


🚀 步骤 2/5: 文本预处理...
✅ 预处理完成（直通）。文件数: 10


### 🔍 查看预处理结果


### 💡 LLM内容过滤说明

**这一步的作用：**
- 输入：嵌入相似度筛选出的段落（如20个段落）
- 处理：使用Nous-Hermes-Llama2-13B模型判断每个段落是否真正与表面化学反应相关
- 输出：进一步筛选的相关段落（如10个段落）

**模型选择：**
- 优先使用：Nous-Hermes-Llama2-13B-Instruct（更智能，性能更好）
- 回退模型：nous-hermes-llama2-13b（稳定可靠）

**为什么段落数会减少：**
- 嵌入相似度只是基于关键词匹配
- LLM过滤会进行更智能的内容理解
- 最终保留真正相关的段落


## 🔍 步骤 3: 嵌入和相似度计算


In [9]:
print("🚀 步骤 3/5: 嵌入和相似度计算...")
print("=" * 50)

from core.embedding import run_embedding_selection

embedding_files = []
for i, processed_file in enumerate(processed_files, 1):
    print(f"\n📄 处理文件 {i}/{len(processed_files)}: {os.path.basename(processed_file)}")
    emb = run_embedding_selection(processed_file, top_n=int(os.getenv('FCPD_TOP_N','10')))
    embedding_files.append(emb)
    print(f"  ✅ 生成嵌入筛选文件: {emb}")

# # 归档
# try:
#     engine_name = ENGINE_NAME
#     for emb in embedding_files:
#         base = os.path.splitext(os.path.basename(emb))[0].replace('Embedding_','')
#         dest_dir = os.path.join(main_output_dir, '..', engine_name, base)
#         os.makedirs(dest_dir, exist_ok=True)
#         if os.path.exists(emb):
#             shutil.copy2(emb, os.path.join(dest_dir, os.path.basename(emb)))
# except Exception:
#     pass

print(f"\n🎉 嵌入和相似度计算完成！")


🚀 步骤 3/5: 嵌入和相似度计算...

📄 处理文件 1/10: 0.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/0/Embedding_0.txt

📄 处理文件 2/10: 1.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/1/Embedding_1.txt

📄 处理文件 3/10: 2.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/2/Embedding_2.txt

📄 处理文件 4/10: 3.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/3/Embedding_3.txt

📄 处理文件 5/10: 5.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/5/Embedding_5.txt

📄 处理文件 6/10: 6.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/6/Embedding_6.txt

📄 处理文件 7/10: 7.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/7/Embedding_7.txt

📄 处理文件 8/10: 8.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/8/Embedding_8.txt

📄 处理文件 9/10: 9.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/9/Embedding_9.txt

📄 处理文件 10/10: 10.txt
  ✅ 生成嵌入筛选文件: evaluation/result/local llm finetuned/10/Embedding_10.txt

🎉 嵌入和相似度计算完成！


### 🔍 查看嵌入结果


In [10]:
# 查看嵌入结果
sample_embedding = embedding_files[0]
print(f"📖 查看嵌入文件: {os.path.basename(sample_embedding)}")
print("=" * 50)

with open(sample_embedding, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()
    
print(f"📊 嵌入相似度筛选后段落数: {len(lines)}")
print("\n📄 相似度最高的段落预览:")
print("-" * 30)
for i, line in enumerate(lines[:3]):
    if line.strip():
        print(f"段落 {i+1}: {line[:200]}..." if len(line) > 200 else f"段落 {i+1}: {line}")


📖 查看嵌入文件: Embedding_0.txt
📊 嵌入相似度筛选后段落数: 30

📄 相似度最高的段落预览:
------------------------------
段落 1: ABSTRACT: In this work, continuous flow nitration of trifluoromethoxybenzene (TFMB) was conducted in a microchannel reactor. The eﬀects of process parameters, including temperature, residence time, su...
段落 3: of the workup. Apart from those substantial advantages presented above, with the numbering-up strategy, the small scaling eﬀect of microreactor makes the laboratory results easier to industrialize.9 G...


## 🤖 步骤 4: LLM内容过滤


In [11]:
print("🚀 步骤 4-5/5: 推理与归档 (本地=五步法 / 在线=直达)...")
print("=" * 50)

final_outputs = {
    'filter': [],
    'summarized': [],
    'summarized_overall': [],
    'impact_analysis': [],
}

for i, embedding_file in enumerate(embedding_files, 1):
    print(f"\n📄 处理文件 {i}/{len(embedding_files)}: {os.path.basename(embedding_file)}")

    if ENGINE == 'local':
        # 本地：五步法，基于 Embedding 文件
        res = engine.process_text_file_comprehensive(embedding_file, mode='comprehensive')
    else:
        # 在线：直达模式（fast），直接用原始TXT
        original_txt_file = output_files[i-1]
        res = engine.process_text_file_comprehensive(original_txt_file, mode='fast')

    # 收集
    final_outputs['filter'].append(res.get('filter'))
    final_outputs['summarized'].append(res.get('summarized'))
    final_outputs['summarized_overall'].append(res.get('summarized_overall'))
    final_outputs['impact_analysis'].append(res.get('impact_analysis'))

    # 归档
    try:
        if ENGINE == 'local':
            base = os.path.splitext(os.path.basename(embedding_file))[0].replace('Embedding_','')
        else:
            base = os.path.splitext(os.path.basename(original_txt_file))[0]
        dest_dir = os.path.join(main_output_dir, '..', ENGINE_NAME, base)
        os.makedirs(dest_dir, exist_ok=True)
        for p in res.values():
            if p and os.path.exists(p):
                shutil.copy2(p, os.path.join(dest_dir, os.path.basename(p)))
    except Exception:
        pass

# 简要预览：Overall / Impact
for i, (overall, impact) in enumerate(zip(final_outputs['summarized_overall'], final_outputs['impact_analysis']), 1):
    if overall and os.path.exists(overall):
        with open(overall, 'r', encoding='utf-8', errors='ignore') as f:
            s = f.read()
        print(f"\n🧩 Overall 预览[{i}]:\n", (s[:400] + '...') if len(s) > 400 else s)
    if impact and os.path.exists(impact):
        with open(impact, 'r', encoding='utf-8', errors='ignore') as f:
            s = f.read()
        print(f"\n📊 Impact 预览[{i}]:\n", (s[:400] + '...') if len(s) > 400 else s)

print("\n🎉 推理完成！")

🚀 步骤 4-5/5: 推理与归档 (本地=五步法 / 在线=直达)...

📄 处理文件 1/10: Embedding_0.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 12, 解析成功: 11, 含省略号: 0, 信号=0条数: 0

📄 处理文件 2/10: Embedding_1.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 7, 解析成功: 7, 含省略号: 0, 信号=0条数: 0

📄 处理文件 3/10: Embedding_2.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 9, 解析成功: 9, 含省略号: 0, 信号=0条数: 0

📄 处理文件 4/10: Embedding_3.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 8, 解析成功: 8, 含省略号: 0, 信号=0条数: 0

📄 处理文件 5/10: Embedding_5.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 8, 解析成功: 8, 含省略号: 0, 信号=0条数: 0

📄 处理文件 6/10: Embedding_6.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 2, 解析成功: 2, 含省略号: 0, 信号=0条数: 0

📄 处理文件 7/10: Embedding_7.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列: content, 段落总数: 3, 解析成功: 3, 含省略号: 0, 信号=0条数: 0

📄 处理文件 8/10: Embedding_8.txt
  ⏩ 跳过 Abstract 步骤 (优化)
  🔎 Summarize统计 -> 段落来源列

In [12]:
# # 兜底：若未初始化 engine，则快速初始化一次
# if 'engine' not in globals():
#     import os, yaml
#     from core.local_pipeline import LocalPipeline
#     from core.models.qwen_llm import QwenLLM
#     from core.models.gemini_llm import GeminiLLM
#     from core.processor import UnifiedTextProcessor

#     cfg = yaml.safe_load(open('config.yaml','r',encoding='utf-8')) or {}
#     eng = (os.getenv('NB_ENGINE') or cfg.get('engine','local')).strip().lower()
#     if eng == 'local':
#         lc = cfg.get('local_model',{}) or {}
#         engine = LocalPipeline(
#             model_name=None,
#             model_path=lc.get('path','models/'),
#             filter_model=lc.get('filter'),
#             abstract_model=lc.get('abstract'),
#             summarize_model=lc.get('summarize'),
#             overall_model=lc.get('overall'),
#             impact_model=lc.get('impact'),
#             finetuned_trigger_name=lc.get('finetuned_trigger_name','My_Finetuned_Model'),
#         )
#     elif eng == 'qwen':
#         q = cfg.get('qwen_api',{}) or {}
#         llm = QwenLLM(api_key_env_var=q.get('api_key_env_var','QWEN_API_KEY'), model_name=q.get('model_name','qwen-plus'))
#         engine = UnifiedTextProcessor(llm)
#     else:
#         g = cfg.get('gemini_api',{}) or {}
#         llm = GeminiLLM(api_key_env_var=g.get('api_key_env_var','GOOGLE_API_KEY'), model_name=g.get('model_name','gemini-1.5-flash'))
#         engine = UnifiedTextProcessor(llm)

# import os, pandas as pd

# # 1) 指定你的 Summarized 文件（每行一个 JSON）
# summarized_file = 'evaluation/result/local llm finetuned/1/Embedding_1_Summarized.txt'

# # 2) 读取为 DataFrame 的 summarized 列
# with open(summarized_file, 'r', encoding='utf-8', errors='ignore') as f:
#     lines = [ln.strip() for ln in f if ln.strip()]
# df_summarized = pd.DataFrame(lines, columns=['summarized'])

# # 3) 调用合并，仅执行 Overall（会使用优化后的微调合并提示词）
# overall_json = engine.summarize_document_overall(df_summarized)

# # 4) 保存
# overall_file = summarized_file.replace('_Summarized.txt', '_Overall.txt')
# with open(overall_file, 'w', encoding='utf-8') as f:
#     f.write(overall_json)

# print('✅ Overall 写入:', overall_file)